# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
#!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

!wget http://1.bp.blogspot.com/-t7VZKyYIyzk/UwGp54KlJkI/AAAAAAAAD9g/gtfiIA5hHKI/s1600/three-musicians-by-pablo-picasso-1342648771_b.jpg

# Imagen para contenido
#!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

!wget https://upload.wikimedia.org/wikipedia/ru/7/7f/The_Beatles_-_Abbey_Road.jpg



# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2020-12-29 20:28:07--  http://1.bp.blogspot.com/-t7VZKyYIyzk/UwGp54KlJkI/AAAAAAAAD9g/gtfiIA5hHKI/s1600/three-musicians-by-pablo-picasso-1342648771_b.jpg
Resolving 1.bp.blogspot.com (1.bp.blogspot.com)... 74.125.199.132, 2607:f8b0:400e:c02::84
Connecting to 1.bp.blogspot.com (1.bp.blogspot.com)|74.125.199.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65135 (64K) [image/jpeg]
Saving to: ‘three-musicians-by-pablo-picasso-1342648771_b.jpg’

three-musicians-by- 100%[===================>]  63.61K  --.-KB/s    in 0s      

2020-12-29 20:28:07 (128 MB/s) - ‘three-musicians-by-pablo-picasso-1342648771_b.jpg’ saved [65135/65135]

--2020-12-29 20:28:07--  https://upload.wikimedia.org/wikipedia/ru/7/7f/The_Beatles_-_Abbey_Road.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [ ]:
%tensorflow_version 1.x   # en Collab

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x   # en Collab`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

Using TensorFlow backend.


In [ ]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/The_Beatles_-_Abbey_Road.jpg")
style_reference_image_path = Path("/content/three-musicians-by-pablo-picasso-1342648771_b.jpg")
result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

total_variation_weight: regula el ruido que tendrá la imagen final de forma de suavizar su presencia. En definitiva, representa el peso de la loss de suavizado.

content_weight: determina el peso que tendrá la imagen de contenido en la imagen de salida. Mientras mayor sea el peso, mayor importancia se le dará a la extracción de características de la imagen de contenido para que sean transferidas a la imagen final. En definitiva, representa el peso de la Loss de Content dentro de la funcion de pérdida total.

style_weight: determina la importancia que tendrá la imagen de estilo en la imagen de salida. Mientras mayor sea el peso, mayor importancia se le dará a la extracción de características de la imagen de estilo para que sean transferidas a la imagen final. En definitiva, representa el peso de la Loss de Style dentro de la funcion de pérdida total.


In [ ]:
total_variation_weight = 0.1
style_weight = 1
content_weight = 100

In [ ]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

La siguiente celda se encarga del preprocesamiento de las imágenes de forma tal de darle a las mismas el formato que necesita la red, realizando para tal fin las siguientes tareas:

-load_img: carga la imagen y le da el tamaño definido por la dimension img_nrows x img_ncols

-img_to_array: convierte la imagen cargada en un array que tendrá en cuenta la cantidad de canales, en este caso, 3. De esta forma el array tendrá la dimensión img_nrows x img_ncols x 3

-expand_dims: Dado que Keras procesa imágenes en batch, expand_dims se encarga de sumarle una dimension al array (1 x img_nrows x img_ncols x 3)

-preprocess_input: convierte el array de RGB a BGR siendo este el formato que espera la VGG19. Dado que la VGG19 se entrenó con el dataset de ImageNet, en esta tarea se realiza una normalizacion de los canales restando la media de los valores de dicho dataset.

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:
La siguiente celda se encarga de deprocesar las imágenes siguiendo la inversa de lo que se hizo en la celda anterior mediante estas tareas:

-Transformación de la imagen en un array de dimension img_rows x img_cols x 3, es decir, el array ya no tiene en cuenta la componente batch size y solo refleja alto,ancho, canales.

-Desnormalización de  cada canal sumándole la media de cada uno de los canales del dataset ImageNet

-Conversión de las imagenes de RGB a BGR

-Nos aseguramos que los valores estén comprendidos entre 0 y 255.

In [ ]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [ ]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [ ]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])


80142336/80134624 [==============================] - 2s 0us/step
Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

La matriz de Gram es aquella obtenida por el producto punto entre los vectores de las activaciones de los filtros de forma tal que puede reflejar la correlación que hay entre los filtros. Si el producto escalar entre los vectores es grande, se dice que los filtros están correlacionados y si es pequeño, los filtros no están correlacionados. La matriz de Gram es usada para buscar las coincidencias entre las correlaciones de la imagen de estilo y la imagen de salida. Esto se logra minimizando la distancia media cuadrática de las entradas de la matriz de Gram de la imagen de estilo original y la imagen de salida

El cálculo de la Matriz de Gram necesita trabajar con matrices de dos dimensiones. Mediante el comando batch_flatten logramos aplanar una matriz de n dimensiones a una matriz de dos dimensiones. Así, por ejemplo, si x tuviera una dimension de 56x56x256, donde 256 se refiere a la cantidad de filtros, con el comando permute_dimensions x pasaría a tener una dimension de 256x56x56 que finalmente, gracias al batch_flatten, la dimensión terminará siendo 256x3136. De esta forma, la función permute_dimension  nos servirà para organizar las matrices antes de aplanarlas.

In [ ]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

style_loss: es la función de pérdida de estilo. Nos indica qué tan similar es la imagen de salida a la imagen se estilo. La función de pérdida entre el estilo de la imagen de estilo y la imagen de salida se calcula como el cuadrado de la diferencia entre la matriz de Gram de la imagen de estilo y la matriz de Gram de la imagen de salida.

content_loss: representa la función de pérdida de contenido. Para el cálculo de la pérdida de contenido  se usa función de error cuadrado que mide la diferencia en las características del contenido entre la imagen de salida y la imagen del contenido

total_variation_loss: calcula la variación entre los valores de píxeles vecinos de la imagen de salida buscando suavizar el ruido , ya sea por presencia de pixeles muy brillantes u oscuros, de forma tal que los valores de los píxeles vecinos sean lo más parecidos posible.

In [ ]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [ ]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [ ]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [ ]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

Evaluator es una clase que calcula el valor de las pérdidas y el valor de los gradientes en un solo paso para evitar que se calculen de forma independiente y así poder lograr un cálculo mas eficiente.  Para obtener los valores se usan dos funciones: por un lado la función loss y, por otro lado, la función  grads. Se usan dos funciones por separado para pérdidas y para gradientes porque asi lo requiere el optimizador L-BFGS que se utiliza para minimizar la funcion de costo. Dicho optimizador, el cual proviene de la librería SciPy y lo invocamos con la función fmin_l_bfgs_b, trabaja con vectores planos y, por esta razon, es que utilizamos el flatten dentro de la función eval_loss_and_ grads que devuelve los valores para las pérdidas y los gradientes.

El paper de Gatys propone el algoritmo BFGS como optimizador. Dicho algoritmo usa la segunda derivada (hessiana) para minimizar la función de costo. Como la segunda derivada es costosa de calcular, se implementa el algoritmo BFGS de memoria limitada L-BFGS el cual utiliza una aproximación del hessiano.

Como alternativa al uso del optimizador L-BFGS, se pueden mencionar otros como ADAM sin embargo siempre resulta mejor posicionado L-BFGS. En esta URL https://blog.slavv.com/picking-an-optimizer-for-style-transfer-86e7b8cba84b es posible vizualizar una comparativa de la perfomance de L-BFGS contra otros optimizadores   

In [ ]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 21859017000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 14s
Start of iteration 1
Current loss value: 14603633000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 4s
Start of iteration 2
Current loss value: 11776477000.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 4s
Start of iteration 3
Current loss value: 10405878000.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 4s
Start of iteration 4
Current loss value: 9563857000.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 4s
Start of iteration 5
Current loss value: 9007344000.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 4s
Start of iteration 6
Current loss value: 8587730400.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 4s
Start of iteration 7
C

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r "/content/drive/MyDrive/generadas.zip" "/content/output"
#files.download("generadas.zip")

  adding: content/output/ (stored 0%)
  adding: content/output/output_at_iteration_68.png (deflated 0%)
  adding: content/output/output_at_iteration_9.png (deflated 0%)
  adding: content/output/output_at_iteration_36.png (deflated 0%)
  adding: content/output/output_at_iteration_14.png (deflated 0%)
  adding: content/output/output_at_iteration_98.png (deflated 0%)
  adding: content/output/output_at_iteration_21.png (deflated 0%)
  adding: content/output/output_at_iteration_20.png (deflated 0%)
  adding: content/output/output_at_iteration_53.png (deflated 0%)
  adding: content/output/output_at_iteration_65.png (deflated 0%)
  adding: content/output/output_at_iteration_51.png (deflated 0%)
  adding: content/output/output_at_iteration_28.png (deflated 0%)
  adding: content/output/output_at_iteration_67.png (deflated 0%)
  adding: content/output/output_at_iteration_78.png (deflated 0%)
  adding: content/output/output_at_iteration_82.png (deflated 0%)
  adding: content/output/output_at_iter

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:
En las imagenes observaremos el resultado que nos arrojó la red para distintas combinaciones de losses. Generaremos una secuencia de tres imágenes que van poniendo énfasis desde el contenido puro hasta poner mucho énfasis en el estilo (se adjuntan las imagenes de salida 8A, 8B y 8C).

Así, en la primer imagen 8A la red basicamente no está resolviendo el estilo para nada. Lo que queda es  esencialmente el contenido teniendo en cuenta que usamos una relacion contenido/estilo de 100/1 siendo 100 el peso que configuramos para el contenido y 1 el peso para el estilo. En la segunda imagen 8B el mayor peso lo pasa a tener el estilo, pero tambien aumentamos el peso del contenido resultando en un mejor balance en la relacion contenido/estilo (40/100) y obteniendo como resultado una imagen donde ya se puede ver cómo es posible fusionar el contenido con el estilo. En la tercer imagen 8C, la red deja de lado el contenido y esencialmente resuelve el estilo con una relacion 1/100, así lo que queda es una imagen donde se pone prácticamente todo el énfasis en el estilo.

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

Se adjuntan las siguientes imágenes:

Style Transfer 1

-Imagen de contenido: 9C1

-Imagen de estilo: 9E1

-Imagen de salida: 9S1


Style Transfer 2

-Imagen de contenido: 9C2

-Imagen de estilo: 9E2

-Imagen de salida: 9S2